This notebook is for our first Classification model. We will start with a basic CNN model just to test that our structure is set up properly. This file may be adjusted and/or replaced before we turn in Milestone 3.

In [12]:
# Install libraries
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install sklearn


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of

In [15]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, concatenate, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
# Pull in datasets
def Data_Set_Init():
    # TODO: write code to pull in datasets of RGB and RGBD data
    # TODO: write code to store all of the labels of each dataset along with the data
    rgbd_images = np.concatenate((rgb_images, depth_images) axis=-1)
    data = rgbd_images, labels
    return data, labels

data, labels = Data_Set_Init()

In [ ]:
# Normalize image data
data = data / 255.0

In [ ]:
# Split the data into test and train
X, y, X_test, y_test = train_test_split(data, labels, test_size=0.2, random_state=983) # X represents RGBD data, y represents labels

In [ ]:
# One-Hot encode the labels
y = tf.keras.utils.to_categorical(y, num_classes=67) # Change to the number of classes of images we are using. Default set contains 67

In [ ]:
# Creating and running the Basic Model

model = Sequential([
    InputLayer(input_shape=(64, 64, 4)), # Adjust the input shape to match the input of the images we decide on (e.g. 64 x 64 pixels)
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(67, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy']) # change to 'sparse_categorical_crossentropy' if we decide against one-hot encoding

history = model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')
